## Jupyter notebook for testing an image

Code from [tests/test_snapshot.py](tests/test_snapshot.py)

required packages 

1. Tensorflow

2. Other packages
```
pip install tqdm chainer
```

In [1]:
import numpy as np
import copy
import os
import sys
import math
from tqdm import tqdm
from scripts import dataset
import scripts.regressionnet
from scripts.regressionnet import batch2feeds, calculate_metric
from chainer import iterators

from scripts import config

/home/yyoo/tf3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def get_gt_data(test_iterator):
    test_it = copy.copy(test_iterator)
    num_test_examples = len(test_it.dataset)
    num_batches = int(math.ceil(num_test_examples / test_it.batch_size))
    gt_joints = list()
    gt_joints_is_valid = list()
    orig_bboxes = list()

    print(len(test_it.dataset))
    for i, batch in tqdm(enumerate(test_it), total=num_batches):
        feeds = batch2feeds(batch)
        gt_joints.append(feeds[1])
        gt_joints_is_valid.append(feeds[2])
        orig_bboxes.append(np.vstack([x['bbox'] for x in feeds[3]]))

    gt_joints = np.vstack(gt_joints)
    gt_joints_is_valid = np.vstack(gt_joints_is_valid)
    orig_bboxes = np.vstack(orig_bboxes)
    return gt_joints, gt_joints_is_valid, orig_bboxes

In [3]:
def main(dataset_name, snapshot_path):
    """
    Args:
        dataset_name: 'mpii' or 'lsp'.
        init_snapshot_path: path to the snapshot to test
    """
    if dataset_name == 'mpii':
        TEST_CV_FILEPATH = os.path.join(config.MPII_DATASET_ROOT, 'test_joints.csv')
        IMG_PATH_PREFIX = os.path.join(config.MPII_DATASET_ROOT, 'images')
        symmetric_joints = "[[12, 13], [11, 14], [10, 15], [2, 3], [1, 4], [0, 5]]"
        ignore_label = -100500
    else:
        TEST_CV_FILEPATH = os.path.join(config.LSP_DATASET_ROOT, 'test_joints.csv')
#        IMG_PATH_PREFIX = ''
        IMG_PATH_PREFIX = os.path.join(config.LSP_DATASET_ROOT, 'images')
        symmetric_joints = "[[8, 9], [7, 10], [6, 11], [2, 3], [1, 4], [0, 5]]"
        ignore_label = -1

    print(TEST_CV_FILEPATH)
    print(IMG_PATH_PREFIX)

    test_dataset = dataset.PoseDataset(
        TEST_CV_FILEPATH,
        IMG_PATH_PREFIX, 227,
        fliplr=False, rotate=False,
        shift=None,
        bbox_extension_range=(1.0, 1.0),
        coord_normalize=True,
        gcn=True,
        fname_index=0,
        joint_index=1,
        symmetric_joints=symmetric_joints,
        ignore_label=ignore_label,
        should_return_bbox=True,
        should_downscale_images=True,
        downscale_height=400
    )

    print("data ready")

    test_iterator = iterators.MultiprocessIterator(
        test_dataset, batch_size=128,
        repeat=False, shuffle=False,
        n_processes=1, n_prefetch=1)

    print("start testing")

    test_net(test_dataset, test_iterator, dataset_name, snapshot_path)

In [4]:
def test_net(test_dataset, test_iterator, dataset_name, snapshot_path):
    if dataset_name not in ['lsp', 'mpii']:
        raise ValueError('Unknown dataset: {}'.format(dataset_name))

    net, loss_op, pose_loss_op, train_op = scripts.regressionnet.create_regression_net(
        n_joints=16 if dataset_name == 'mpii' else 14,
        init_snapshot_path=snapshot_path,
        is_resume=True,
        net_type='Alexnet',
        optimizer_type='momentum',
        gpu_memory_fraction=0.32)  # Set how much GPU memory to reserve for the network
    print(snapshot_path)
    for ext in np.linspace(1.0, 2.0, 6, True):
        print('\n====================')
        print('BBOX EXTENSION:', ext)
        test_dataset.bbox_extension_range = (ext, ext)
        scripts.regressionnet.evaluate_pcp(net, pose_loss_op, test_iterator, None,
                                           dataset_name=dataset_name,
                                           tag_prefix='test')

In [5]:
dataset_name = 'lsp'
snapshot_path = 'out/lsp_alexnet_imagenet/checkpoint-1000000'

# init_snapshot_path = os.path.join(config.ROOT_DIR, 'out/mpii_alexnet_imagenet/checkpoint-10000')
# init_snapshot_path = os.path.join(config.ROOT_DIR, 'out/lsp_alexnet_scratch/checkpoint-10000')
main(dataset_name, snapshot_path)

76it [00:00, 755.40it/s]

/var/data/lsp/test_joints.csv
/var/data/lsp/images
Reading dataset from /var/data/lsp/test_joints.csv
Downscale images to the height 400px


1000it [00:00, 1581.16it/s]


Joints shape: (14, 2)
data ready
start testing
Alexnet::__create_architecture()
Initializing conv1 with random
Initializing conv2 with random
Initializing conv3 with random
Initializing conv4 with random
Initializing conv5 with random
Initializing fc6 with random
Initializing fc7 with random
Initializing fc8 with random
Initializing layer 99 with random
Restoring everything from snapshot and resumingout/lsp_alexnet_imagenet/checkpoint-1000000
INFO:tensorflow:Restoring parameters from out/lsp_alexnet_imagenet/checkpoint-1000000


INFO:tensorflow:Restoring parameters from out/lsp_alexnet_imagenet/checkpoint-1000000


...

Step 1000000 test/pose_loss = 0.020
lsp
Step	 1000000	 test/mPCP	 0.534
Step 1000000 test/parts_PCP:
Head	Torso	U Arm	L Arm	U Leg	L Leg	mean
0.517	0.930	0.396	0.169	0.700	0.492	0.534
lsp
Step	 1000000	 test/mRelaxedPCP	 0.636
Step 1000000 test/parts_RelaxedPCP:
Head	Torso	U Arm	L Arm	U Leg	L Leg	mean
0.619	0.961	0.514	0.279	0.811	0.633	0.636
lsp


  0%|          | 0/8 [00:00<?, ?it/s]

Step	 1000000	 test/mPCKh	 0.439
Step	 1000000	 test/mSymmetricPCKh	 0.461
Step 1000000 test/parts_SymmetricPCKh:
Head	Neck	Shoulder	Elbow	Wrist	Hip	Knee	Ankle
0.548	0.677	0.546	0.369	0.239	0.647	0.444	0.217

BBOX EXTENSION: 2.0
1000


100%|██████████| 8/8 [00:04<00:00,  1.83it/s]


Step 1000000 test/pose_loss = 0.019
lsp
Step	 1000000	 test/mPCP	 0.533
Step 1000000 test/parts_PCP:
Head	Torso	U Arm	L Arm	U Leg	L Leg	mean
0.519	0.930	0.393	0.165	0.698	0.492	0.533
lsp
Step	 1000000	 test/mRelaxedPCP	 0.635
Step 1000000 test/parts_RelaxedPCP:
Head	Torso	U Arm	L Arm	U Leg	L Leg	mean
0.619	0.962	0.510	0.278	0.809	0.633	0.635
lsp
Step	 1000000	 test/mPCKh	 0.439
Step	 1000000	 test/mSymmetricPCKh	 0.460
Step 1000000 test/parts_SymmetricPCKh:
Head	Neck	Shoulder	Elbow	Wrist	Hip	Knee	Ankle
0.547	0.673	0.549	0.367	0.239	0.646	0.445	0.216
